In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss

import sys
sys.path.insert(0,"libs")
import data_cleaning as dc

In [2]:
train = pd.read_csv("data/train.csv")

train = dc.massage_df(train)
train = train[train.AnimalType=="Cat"]

In [3]:
# split all available data into test and train subsets
frac = .6 # use 60% data for training
test_rows = np.random.choice(train.index.values
            ,int(round( len(train)*frac )),replace=False )
mytrain = train.ix[test_rows]
mytest  = train.drop(test_rows)

In [18]:
basic_features = ['age_numeric_years', 'neuter_status', 'sex'
                  , 'mixed',"has_name","is_weekend"]
cat_features   = ['has_Lynx', 'has_Brown', 'has_White', 'has_Orange',
   'has_Tabby', 'has_Tan', 'has_Gray', 'has_Calico', 'has_Blue',
   'has_Agouti', 'has_Lilac', 'has_Tortie', 'has_Black', 'has_Point',
   'catbreed']
time_features = ['month', 'day_of_month', 'day_of_year',
   'day_of_week', 'day_of_week_type', 'season' ]

all_features = basic_features# + cat_features + time_features

In [19]:
#features = ["neuter_status","age_numeric_days","day_of_week_type"]
#features = np.random.choice(all_features,2,replace=False)
#print(features)
features = basic_features

mytrain_features = pd.get_dummies(mytrain[features])
print("database utilization = %f" % \
      (len( mytrain_features.dropna() )/len(mytrain_features)) )
mytrain_features = mytrain_features.dropna()
mytrain_outcomes = mytrain.OutcomeType.ix[mytrain_features.index]
mytest_features = pd.get_dummies(mytest[features]).dropna()
mytest_outcomes = mytest.OutcomeType.ix[mytest_features.index]

# train
#classifier = LogisticRegression()
classifier = tree.DecisionTreeClassifier(max_depth=3)
classifier.fit(mytrain_features,mytrain_outcomes);

predictions = classifier.predict(mytest_features)
prediction_probabilities = classifier.predict_proba(mytest_features)

# check performance
acc = accuracy_score(mytest_outcomes, predictions)
los = log_loss( pd.get_dummies( mytest_outcomes ).values, prediction_probabilities)
print("accuracy = {0:1.2f} \nlog-loss = {1:1.3f}".format(acc,los) )

database utilization = 0.000000
accuracy = 0.75 
log-loss = 0.710
